<a href="https://colab.research.google.com/github/anshuiitb/pytorch/blob/main/CNN_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
%matplotlib inline

from torchvision.utils import make_grid

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix


In [27]:
# convert mnist Image file in tensor of 4-Dimension
transforms = transforms.ToTensor()

In [28]:
# train data

train_data = datasets.MNIST(root='/cnn_data', train=True, download=True, transform=transforms)

In [29]:
test_data = datasets.MNIST(root='/cnn_data', train=False, download=True, transform=transforms)

In [30]:
train_loader = DataLoader(train_data , batch_size = 10 , shuffle = True)
test_loader = DataLoader(test_data , batch_size = 10 , shuffle = False)

In [33]:
# define our cnn model
conv1 = nn.Conv2d( 1, 6 , 3 ,1)
conv2 = nn.Conv2d( 6 , 16 , 3 ,1)

In [32]:
#grad 1 Mnist image

Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))

In [34]:
for i , (X_train , y_train ) in enumerate(train_data):
  break

In [35]:
X_train.shape

torch.Size([1, 28, 28])

In [44]:
x = X_train.view(1,1,28,28)

In [45]:
x = F.relu(conv1(x))


In [46]:
x.shape

torch.Size([1, 6, 26, 26])

In [47]:
x = F.max_pool2d(x,2,2)

In [48]:
x.shape

torch.Size([1, 6, 13, 13])

In [49]:
# second conv layer
x = F.relu(conv2(x))

In [50]:
x.shape

torch.Size([1, 16, 11, 11])

In [51]:
x = F.max_pool2d(x,2,2)

In [52]:
x.shape

torch.Size([1, 16, 5, 5])

In [64]:
class cnn(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d( 1, 6 , 3 ,1)
    self.conv2 = nn.Conv2d( 6 , 16 , 3 ,1)
    # fully connected layer
    self.fc1 = nn.Linear(5*5*16 , 120)
    self.fc2 = nn.Linear(120 , 84)
    self.fc3 = nn.Linear(84 , 10)


  def forward(self, X):
    X = F.relu(self.conv1(X))
    X = F.max_pool2d(X,2,2)
    X = F.relu(self.conv2(X))
    X = F.max_pool2d(X,2,2)
    X = X.view(-1 , 5*5*16)

    X = F.relu(self.fc1(X))
    X = F.relu(self.fc2(X))
    X = self.fc3(X)

    return F.log_softmax(X, dim=1)

In [65]:
torch.manual_seed(41)
model=cnn()
model

cnn(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [66]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [69]:
import time
start_time = time.time()


epochs = 5
train_losses =[]
test_losses = []
train_correct = []
test_correct = []

for i in range(epochs):
  trn_corr = 0
  tst_corr = 0


  #train
  for b , (X_train , y_train) in enumerate(train_loader):
    b+=1
    y_pred = model(X_train)
    loss = criterion(y_pred , y_train)
    predicted = torch.max(y_pred.data , 1)[1]
    batch_corr = (predicted == y_train).sum()
    trn_corr+= batch_corr


    #update para
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    #print out some resuts
    if b%600 ==0:
      print(f'epoch : {i} , batch : {b} , loss : {loss.item()}')

    train_losses.append(loss)
    train_correct.append(trn_corr)

  #test
  with torch.no_grad():
    for b , (X_test , y_test) in enumerate(test_loader):
      y_val = model(X_test)
      predicted = torch.max(y_val.data , 1)[1]
      tst_corr = (predicted == y_test).sum()

    loss = criterion(y_val , y_test)

    test_losses.append(loss)
    test_correct.append(tst_corr)



current_time = time.time()
total = current_time - start_time
print(total)

epoch : 0 , batch : 600 , loss : 0.14749222993850708
epoch : 0 , batch : 1200 , loss : 0.2706121802330017
epoch : 0 , batch : 1800 , loss : 0.00956921186298132
epoch : 0 , batch : 2400 , loss : 0.0063883536495268345
epoch : 0 , batch : 3000 , loss : 0.002237545093521476
epoch : 0 , batch : 3600 , loss : 0.23918752372264862
epoch : 0 , batch : 4200 , loss : 0.00048339104978367686
epoch : 0 , batch : 4800 , loss : 0.1504632532596588
epoch : 0 , batch : 5400 , loss : 0.0010857878951355815
epoch : 0 , batch : 6000 , loss : 0.001957091735675931
epoch : 1 , batch : 600 , loss : 0.011673185043036938
epoch : 1 , batch : 1200 , loss : 0.13187162578105927
epoch : 1 , batch : 1800 , loss : 0.5180457830429077
epoch : 1 , batch : 2400 , loss : 0.31928375363349915
epoch : 1 , batch : 3000 , loss : 0.5415772795677185
epoch : 1 , batch : 3600 , loss : 0.004114631563425064
epoch : 1 , batch : 4200 , loss : 0.4243695139884949
epoch : 1 , batch : 4800 , loss : 0.0007736948318779469
epoch : 1 , batch : 54